## Population decoding
Population decoding of the object identity and modality condition based on support vector machine.
The purpose of this analysis is to see how much decoding performance improves when temporal information is added sequentially.

In [7]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm

import pingouin as pg

import matplotlib as mpl
import matplotlib.pyplot as plt  

from datetime import date
import time

import h5py

In [9]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [10]:
mother_path = Path('D:/Multi-modal project/')
fig_path = mother_path / 'analysis' / 'result' / 'figures'

### Parameter setting

In [11]:
# colors for multimodal, vis-only, aud-only conditions
color = ['tab:purple','tab:blue','tab:red','tab:green']
color2 = ['cyan','magenta','gray']
linestyle = ['-',':']

today = str(date.today())

### Data preparation

In [12]:
cell_path = mother_path/'analysis'/'result'/'zFR export'/'13-Jun-2023'
cell_list = os.listdir(cell_path)

# load hdf5 files containing shuffled results
data_path = mother_path /'analysis'/'result'/'3. ANOVA'/'2023-07-10'
data2_path = mother_path /'analysis'/'result'/'3.1 Multiple linear regression'/'2023-07-10'
f = h5py.File(data_path/'2023-07-10_ANOVA_result.hdf5','r')
f2 = h5py.File(data2_path/'2023-07-10_multiple_regression_result.hdf5','r')

In [13]:
def h5_to_df(f):
    keys = list(f.keys())
    attributes = {}
    
    for key in keys:
        attributes[key] = dict(f[key].attrs.items())
    
    df = pd.DataFrame.from_dict(attributes, orient='index')
    df.index.name = 'key'
    df = df.reset_index()
    
    return df

In [18]:
df = h5_to_df(f)
df = df[df['region']=='PER']
df.reset_index(inplace=True, drop=True)

for key in df['key']:
    if df.loc[df['key']==key, 'object cell'].item() == 0:
        df.loc[df['key']==key, 'category'] = 'none'
    else:
        if f2[key].attrs['response cell'] == 0:
            df.loc[df['key']==key, 'category'] = 'object'
        else:
            df.loc[df['key']==key, 'category'] = 'response'

### Check the number of neurons available for population analysis

In [20]:
rat_list = np.sort(df['rat'].unique())
print(rat_list)

['600' '602' '640' '647' '654' '679' '699']


In [21]:
# none cell + object cell (response cell excluded)
cell_array = np.zeros((len(rat_list),int(df['session'].max())))
for i in range(len(df)):
    rat = df.loc[i, 'rat']
    rat_id = np.where(rat_list==rat)[0][0]
    ss = int(df.loc[i, 'session'])
    
    if df.loc[i,'category'] != 'response':
        cell_array[rat_id, ss-1] += 1

print('===== Object cells + None =====')        
print(cell_array)

===== Object cells + None =====
[[ 8. 11.  3. 11.  7.  0.  0.]
 [26. 19. 28. 16. 16.  0.  0.]
 [10. 13.  9.  4.  5.  0.  0.]
 [ 0.  0.  0. 15. 10.  8.  4.]
 [ 7.  4.  0.  0.  1.  0.  0.]
 [ 0.  2.  2.  3.  0.  1.  0.]
 [ 9.  9. 16. 15. 17.  7.  0.]]


In [22]:
# only object cell (response cell, none cell excluded)
cell_array = np.zeros((len(rat_list),int(df['session'].max())))
for i in range(len(df)):
    rat = df.loc[i, 'rat']
    rat_id = np.where(rat_list==rat)[0][0]
    ss = int(df.loc[i, 'session'])
    
    if df.loc[i,'category'] == 'object':
        cell_array[rat_id, ss-1] += 1

print('====== Only object cells ======')
print(cell_array)

====== Only object cells ======
[[ 5.  6.  2.  4.  5.  0.  0.]
 [15.  6. 10.  5.  8.  0.  0.]
 [ 7.  9.  4.  3.  3.  0.  0.]
 [ 0.  0.  0. 10.  4.  4.  4.]
 [ 3.  2.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.]
 [ 4.  3.  6.  3.  8.  3.  0.]]


In [23]:
num_obj = np.sum(cell_array,axis=1)
for i in range(len(rat_list)):
    print(f'LE{rat_list[i]}: {int(num_obj[i])}')

LE600: 22
LE602: 44
LE640: 26
LE647: 22
LE654: 6
LE679: 2
LE699: 27


## Support vector machine

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [224]:
def subsample(df, rat, num_sample, bin_size):
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    
    num_bin = int(90/bin_size)
    object_cell_list = df.loc[(df['rat']==rat)&(df['category']=='object'),'key'].to_numpy()
    
    # M by N array of subsampled firing rates
    # M = stimulus condition * number of subsampled trials
    # N = number of cells * number of time bins
    X = np.zeros((6*num_sample, len(object_cell_list)*num_bin))
    
    for i,cell_key in enumerate(object_cell_list):
    
        # load trial-by-trial firing rate data of a cell
        key = cell_key.zfill(4)
        cell_filename = list(cell_path.glob(f'{key}*.csv'))
        df_cell = pd.read_csv(cell_filename[0])
        df_cell.drop(df_cell[df_cell.Correctness==0].index,inplace=True)
        df_cell.reset_index(inplace=True,drop=True)
        
        # goal location (left or right) of each object condition
        boy_goal = df_cell.loc[df_cell['Visual']=='Boy','RWD_Loc'].values[0]
        egg_goal = df_cell.loc[df_cell['Visual']=='Egg','RWD_Loc'].values[0]

        # stimulus condition (identity x modality)
        cond = [(df_cell['Type']=='Multimodal')&(df_cell['RWD_Loc']==boy_goal),
                (df_cell['Type']=='Visual')&(df_cell['RWD_Loc']==boy_goal),
                (df_cell['Type']=='Auditory')&(df_cell['RWD_Loc']==boy_goal),
                (df_cell['Type']=='Multimodal')&(df_cell['RWD_Loc']==egg_goal),
                (df_cell['Type']=='Visual')&(df_cell['RWD_Loc']==egg_goal),
                (df_cell['Type']=='Auditory')&(df_cell['RWD_Loc']==egg_goal)]
    
        fr_id = df_cell.columns.get_loc('Var10')  # get the index of the first firing rate column        
        FR = df_cell.iloc[:,fr_id:].to_numpy()    # convert firing rate data to numpy array
    
        # recalculate mean firing rates of each time bin
        FR_bin = np.zeros((len(df_cell),num_bin))
        for b in range(num_bin):
            FR_bin[:,b] = FR[:,b*bin_size:(b+1)*bin_size].mean(axis=1)
        
        for c in range(len(cond)):
            # subsample n trials from each stimulus condition
            subsampled_trial = df_cell[cond[c]].sample(num_sample).index
            # firing rates of subsampled trials
            FR_sub = FR_bin[subsampled_trial,:]
            # add subsampled firing rate data to the result array
            X[num_sample*c:num_sample*(c+1), num_bin*i:num_bin*(i+1)] = FR_sub
    
    X_norm = scaler.fit_transform(X)
        
    return X_norm

### Hyperparameter optimization

In [256]:
# pre-determined hyperparameter parameter list
#params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
#               {'kernel': ['linear'], 'C': [1e-5,1e-4,10e-3,10e-2,10e-1,1,10,100,1000,10000]}]
params_grid = [{'kernel': ['linear'], 'C': [1e-8,1e-7,1e-6,1e-5,1e-4,10e-3,10e-2,10e-1,1]}]
grid_search = GridSearchCV(svm.SVC(),params_grid,cv=5,scoring='accuracy')

In [266]:
%%time

rat_list = ['600', '602', '640', '647', '699']

num_iter = 1000
Y = np.repeat(np.arange(6), 5)

best_c = np.zeros((num_iter, len(rat_list)))
best_score = np.zeros((num_iter, len(rat_list)))

for i,rat in enumerate(rat_list):
    loop_start = time.time()
    for iter_run in range(num_iter):
        X = subsample(df, rat, 5, 10)
        grid_search.fit(X, Y)
        best_c[iter_run,i] = grid_search.best_estimator_.C
        best_score[iter_run,i] = grid_search.best_score_
    loop_end = time.time()
    loop_time = divmod(loop_end-loop_start,60)
    print(f'LE{rat} completed /// Elapsed time: {int(loop_time[0])} min {loop_time[1]:.2f} sec')


KeyboardInterrupt: 

In [245]:
# View the accuracy score
print('Best score for training data:', grid_search.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',grid_search.best_estimator_.C,"\n") 
print('Best Kernel:',grid_search.best_estimator_.kernel,"\n")
print('Best Gamma:',grid_search.best_estimator_.gamma,"\n")

Best score for training data: 0.4333333333333333 

Best C: 0.001 

Best Kernel: linear 

Best Gamma: scale 



In [221]:
final_model = grid_search.best_estimator_
Y_pred = final_model.predict(X)

In [ ]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(Y_test_label,Y_pred_label))
print("\n")
print(classification_report(Y_test_label,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(X_train_scaled , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , Y_test ))

svm_model.score

In [ ]:
%%time

num_sample = 1000
result_hyper = np.zeros((num_sample,3))
svm_clf = svm.SVC(kernel='linear')

for sample_run in range(num_sample):
    tic = time.time()
    
    X_M = np.zeros((10,0))
    X_V = np.zeros((10,0))
    X_A = np.zeros((10,0))
    
    for i,cell_id in enumerate(df2.Key):
        if cell_id < 10:
            cell_name = '000'+str(cell_id)
        elif cell_id < 100:
            cell_name = '00'+str(cell_id)
        elif cell_id < 1000:
            cell_name = '0'+str(cell_id)
        else:
            cell_name = str(cell_id)
    
        os.chdir(cell_path)
        cell_filename = glob(cell_name+'*')[0]
    
        x_M = subsample_fr(cell_filename,'Multimodal')
        x_V = subsample_fr(cell_filename,'Visual')
        x_A = subsample_fr(cell_filename,'Auditory')

        X_M = np.c_[X_M,x_M]
        X_V = np.c_[X_V,x_V]
        X_A = np.c_[X_A,x_A]
        
            
    Y = np.r_[np.zeros((5,1)),np.ones((5,1))]
        
    grid_search.fit(X_M,np.transpose(Y)[0])
    M_best = grid_search.best_params_['C']
    grid_search.fit(X_V,np.transpose(Y)[0])
    V_best = grid_search.best_params_['C']
    grid_search.fit(X_A,np.transpose(Y)[0])
    A_best = grid_search.best_params_['C']    

    result_hyper[sample_run,0] = M_best
    result_hyper[sample_run,1] = V_best
    result_hyper[sample_run,2] = A_best